# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298497060761                   -0.85    5.2
  2   -8.300217259610       -2.76       -1.25    1.0
  3   -8.300443669869       -3.65       -1.89    3.4
  4   -8.300462663540       -4.72       -2.78    1.8
  5   -8.300464297349       -5.79       -3.10    3.2
  6   -8.300464446345       -6.83       -3.27    5.0
  7   -8.300464540008       -7.03       -3.42    7.8
  8   -8.300464595287       -7.26       -3.57    1.0
  9   -8.300464626244       -7.51       -3.73    1.9
 10   -8.300464637841       -7.94       -3.89    1.1
 11   -8.300464643455       -8.25       -4.21    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64176481756                   -0.70    6.0
  2   -16.67801315928       -1.44       -1.14    1.4
  3   -16.67920847254       -2.92       -1.88    3.4
  4   -16.67928203727       -4.13       -2.75    5.2
  5   -16.67928603615       -5.40       -3.11    4.9
  6   -16.67928620213       -6.78       -3.52    1.2
  7   -16.67928622079       -7.73       -4.04    1.5


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32709527767                   -0.56    7.2
  2   -33.33451763437       -2.13       -1.00    1.2
  3   -33.33599895433       -2.83       -1.73    8.6
  4   -33.33687130477       -3.06       -2.39    5.1
  5   -33.33693958947       -4.17       -2.97    7.2
  6   -33.33694389640       -5.37       -3.86    4.1
  7   -33.33694392406       -7.56       -4.26    6.2


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298202477923                   -0.85    5.1
  2   -8.300223919555       -2.69       -1.58    1.0
  3   -8.300431778214       -3.68       -2.43    5.6
  4   -8.300334863786   +   -4.01       -2.20    7.5
  5   -8.300463995665       -3.89       -3.38    4.2
  6   -8.300464574137       -6.24       -3.88    2.1
  7   -8.300464635315       -7.21       -4.29    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32565639119                   -0.56    6.6
  2   -33.28080680294   +   -1.35       -1.25    1.6
  3   +11.54700869192   +    1.65       -0.24   10.1
  4   -33.32545264030        1.65       -1.83    6.8
  5   -33.21199601343   +   -0.95       -1.32    6.1
  6   -33.09455611991   +   -0.93       -1.37    4.5
  7   -33.22456836615       -0.89       -1.53    5.6
  8   -33.33592099783       -0.95       -2.38    4.4
  9   -33.33612310609       -3.69       -2.46    2.5
 10   -33.33663363666       -3.29       -2.50    2.8
 11   -33.33693245724       -3.52       -3.12    2.0
 12   -33.33693641015       -5.40       -3.19    2.9
 13   -33.33693874289       -5.63       -3.26    1.2
 14   -33.33694117885       -5.61       -3.45    1.9
 15   -33.33694290108       -5.76       -3.61    1.6
 16   -33.33694365657       -6.12       -3.82    2.1
 17   -33.33694371661       -7.22       -4.08 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.